# Aula 07 - Exercício 06
## KNN e Métricas de avaliação
### Alunos:
 - Antonio Moreira
 

O dataset ["Breast Cancer Wisconsin"](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Original) é um dataset que contém 699 instâncias, cada uma com 10 atributos númericos e 1 atributo correspondente a classe. A idéia do dataset é conseguir classificar um tumor como benigno ou maligno a partir de 10 variáveis descritivas. Foi adicionado um cabeçalho no arquivo original para facilitar seu manuseio.

- Notas:

  - **O dataset possui valores ausentes, representados por "?"**
  - A primeira variável do dataset é o ID de cada paciente. Essa variável **NÃO** deve ser utilizada no classificador 

---

- Carregue o dataset a partir do arquivo fornecido, substituindo os valores ausentes pela média dos valores daquela coluna. Utilize as classes `sklearn.impute.SimpleImputer` e a biblioteca `pandas`.

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

data = pd.read_csv("./datasets/breast-cancer-wisconsin.data", na_values='?')

#data = data.replace('?', np.nan) # substitui valores '?' por NaN para tpdos os atributos serem numéricos

X = data.values[...,1:-1]  # features sem ID
Y = data.values[...,-1]    # classes

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imp_mean.fit_transform(X) # Substituindo NaN pela média da feature



---
- Centralize e normalize os dados e separe 80% do conjunto para treino e 20% para testes. Faça uma divisão dos dados de maneira **estratificada**.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

np.random.seed(42)

# standarizando
scaler = StandardScaler()
X = scaler.fit_transform(X)

# splitando
(X_train, X_test, Y_train, Y_test) = train_test_split(X,Y, test_size=0.2, random_state=42, stratify=Y)



---

- Faça classificação no conjunto de testes utilizando 4 classificadores KNN, com K=3 e 15 e p=1 e 2 (distância de Minkowski/Euclidiana). Para cada classificador, calcule e exiba a matriz de confusão bem como a acurácia do classificador. **Para calcular a acurácia, utilize APENAS a matriz de confusão.**

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

classifier_1 = KNeighborsClassifier(n_neighbors=3, p=1) # 3 vizinhos próxmimos usando distância de manhattan
classifier_1.fit(X_train, Y_train)
y1_pred = classifier_1.predict(X_test)

conf_matrix1 = confusion_matrix(Y_test, y1_pred)
print("Classfier 1: \n", conf_matrix1)
print("Accuracy 1: ", np.trace(conf_matrix1)/conf_matrix1.sum(), '\n')
#------
classifier_2 = KNeighborsClassifier(n_neighbors=3, p=2) # 3 vizinhos próxmimos usando distância euclidiana
classifier_2.fit(X_train, Y_train)
y2_pred = classifier_2.predict(X_test)

conf_matrix2 = confusion_matrix(Y_test, y2_pred)
print("Classfier 2: \n", conf_matrix2)
print("Accuracy 2: ", np.trace(conf_matrix2)/conf_matrix2.sum(), '\n')
#------
classifier_3 = KNeighborsClassifier(n_neighbors=15, p=1) # 15 vizinhos próxmimos usando distância de manhattan
classifier_3.fit(X_train, Y_train)
y3_pred = classifier_3.predict(X_test)

conf_matrix3 = confusion_matrix(Y_test, y3_pred)
print("Classfier 3: \n", conf_matrix3)
print("Accuracy 3: ", np.trace(conf_matrix3)/conf_matrix3.sum(), '\n')
#------
classifier_4 = KNeighborsClassifier(n_neighbors=15, p=2) # 15 vizinhos próxmimos usando distância euclidiana
classifier_4.fit(X_train, Y_train)
y4_pred = classifier_4.predict(X_test)

conf_matrix4 = confusion_matrix(Y_test, y4_pred)
print("Classfier 4: \n", conf_matrix4)
print("Accuracy 4: ", np.trace(conf_matrix4)/conf_matrix4.sum())


Classfier 1: 
 [[88  4]
 [ 5 43]]
Accuracy 1:  0.9357142857142857 

Classfier 2: 
 [[88  4]
 [ 5 43]]
Accuracy 2:  0.9357142857142857 

Classfier 3: 
 [[89  3]
 [ 4 44]]
Accuracy 3:  0.95 

Classfier 4: 
 [[89  3]
 [ 2 46]]
Accuracy 4:  0.9642857142857143




---
Agora vamos analisar um problema de classificação não binário. Para isso, vamos utilizar o conjunto Iris. Repetindo os passos anteriores:
- Carregue o conjunto Iris
- Centralize e normalize os dados
- Separe o conjunto, de maneira **NÃO** estratificada, em 50% treino e 50% teste (a ideia é que o classificador tenha um erro maior)
- Utilize um classificador KNN com K=1 para predizer o conjunto de teste. Essa predição será utilizado posteriormente.


In [4]:
from sklearn.datasets import load_iris

(X, Y) = load_iris(return_X_y=True)

# standarizando
X = scaler.fit_transform(X)

# splitando
(X_train, X_test, Y_train, Y_test) = train_test_split(X,Y, test_size=0.2, random_state=42)

# KNN com 1 vizinho próximo
classifier_iris = KNeighborsClassifier(n_neighbors=1)
classifier_iris.fit(X_train, Y_train)
yiris_pred = classifier_iris.predict(X_test)



---

- Mostre a matriz de confusão do conjunto de teste e em seguida, para cada variável do conjunto, mostre sua matriz de confusão binária.
  - Dica: Pesquise sobre o método `numpy.delete`

In [12]:
conf_matrix = confusion_matrix(Y_test, yiris_pred)

conf_atr = list()

for i in range(len(conf_matrix)):
    row_rmv = np.delete(conf_matrix, i, 0)
    col_rmv = np.delete(row_rmv, i, 1)
    x = np.array([[conf_matrix[i,i], conf_matrix[i].sum()-conf_matrix[i,i]], [conf_matrix[...,i].sum()-conf_matrix[i,i], col_rmv.sum()]])
    conf_atr.append(x)
    print("Confusion Matrix Atr", i,':\n', x, '\n')

Confusion Matrix Atr 0 :
 [[10  0]
 [ 0 20]] 

Confusion Matrix Atr 1 :
 [[ 8  1]
 [ 0 21]] 

Confusion Matrix Atr 2 :
 [[11  0]
 [ 1 18]] 





---

- Utilizando as matrizes de confusão binárias calculadas acima, calcule o **Recall** e **Precisão** de cada variável do conjunto.
  - Dica 1: Lembre-se que ao calcular a matriz de confusão binária para cada classe, obtemos os valores VP, FP, FN e VN.
  - Dica 2: Lembre-se que Recall é calculado por $\frac{VP}{VP+FN}$ e Precisão por  $\frac{VP}{VP+FP}$.

In [19]:
for i in conf_atr:
    print("Recall :", i[0,0]/i[...,0].sum())
    print("Precision: ", i[0,0]/i[0,...].sum())
    print("--------")

Recall : 1.0
Precision:  1.0
--------
Recall : 1.0
Precision:  0.8888888888888888
--------
Recall : 0.9166666666666666
Precision:  1.0
--------




---
Considere o conjunto de dados abaixo, onde são representados um conjunto de $n$ pontos que podem ser classificados em duas classes: verde ou azul. **Todo** o conjunto pode ser visto na imagem abaixo. O ponto preto $q$ é um ponto de consulta, cuja classe é desconhecida. Suponha que foi usado um classificador do tipo KNN com distância euclidiana para classificar esse ponto.


![](https://drive.google.com/uc?export=view&id=1e7oUOHOwyFZ8R1Xy0FPUJ8n_M0YGB2_d)


- Qual a saída desse classificador para K=3?

Para **k=3** o classificador classificará o _query point_ como **azul**, pois são os 3 pontos mais próximos.



---

- E para K=9?

Para **k=9** o classificador encontrará 6 pontos verdes e 3 pontos azuis próximos ao _query point_, portanto classificando como **verde**.



---

 - E para o caso extremo de K=$n$? A posição de $q$ no espaço interfere nesse resultado? Justifique.

Não, pois a grande maioria de pontos neste _dataset_ é verde, logo a maioria de classes "votantes" serão verdes.